In [ ]:
# !pip install -q mediapipe==0.10.0
!pip install --upgrade -q gspread

from google.colab.patches import cv2_imshow
from google.colab import auth
import gspread
import pandas as pd
from torchvision.transforms import RandomResizedCrop, Compose, Normalize, ToTensor
from skimage import io, transform
from tqdm import tqdm
auth.authenticate_user()
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.0 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
print(gc.open('hands and palms loras'))

<Spreadsheet 'hands and palms loras' id:1GHwH5am7o7RKga8FPruSKLgMTXIWG_wEMKk__L0J72A>


In [ ]:
worksheet = gc.open('hands and palms loras').worksheet('scale1').get("A2:I625")
df = pd.DataFrame.from_records(worksheet)
df = df.rename({0:'rank', 1:'checkpoint', 2:'side',3:'gender', 4:'hand', 5:'image', 6: 'bad', 7:'ok', 8:'good'}, axis=1)
df



,rank,checkpoint,side,gender,hand,image
0,64,30000,front,male,right,5
1,64,30000,front,male,right,4
2,64,30000,front,male,right,3
3,64,30000,front,male,right,2
4,64,30000,front,male,right,1
...,...,...,...,...,...,...
571,4,5000,back,female,left,4
572,4,5000,back,female,left,3
573,4,5000,back,female,left,2
574,4,5000,back,female,left,1


In [ ]:
from transformers import TFAutoModelForImageClassification

id2label = {1:'goodhand', 0:'badhand'}
label2id = {v:k for k,v in id2label.items()}

model = TFAutoModelForImageClassification.from_pretrained(
    'DownwardSpiral33/hands_palms_classifier_rot02',
    id2label=id2label,
    label2id=label2id,
    revision='0192149ee2e394132560a65588bea9de138d4d0c', #0.7419354838709677
)

config.json:   0%|          | 0.00/656 [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/343M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFViTForImageClassification.

All the layers of TFViTForImageClassification were initialized from the model checkpoint at DownwardSpiral33/hands_palms_classifier_rot02.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFViTForImageClassification for predictions without further training.


In [ ]:
from transformers import AutoImageProcessor
checkpoint = "google/vit-base-patch16-224-in21k"
image_processor = AutoImageProcessor.from_pretrained(checkpoint)

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

In [12]:
from tqdm import tqdm
from PIL import Image
import tensorflow as tf
import numpy as np
import keras
from tensorflow import keras
from tensorflow.keras import layers


size = (image_processor.size["height"], image_processor.size["width"])

val_data_augmentation = keras.Sequential(
    [
        layers.CenterCrop(size[0], size[1]),
        layers.Rescaling(scale=1.0 / 127.5, offset=-1),
    ],
    name="val_data_augmentation",
)


def preprocess_val(example_batch):
    """Apply val_transforms across a batch."""
    images = [
        val_data_augmentation(convert_to_tf_tensor(image.convert("RGB"))) for image in example_batch["image"]
    ]
    example_batch["pixel_values"] = [tf.transpose(tf.squeeze(image)) for image in images]
    return example_batch

def convert_to_tf_tensor(image: Image):
    np_image = np.array(image)
    tf_image = tf.convert_to_tensor(np_image)
    # `expand_dims()` is used to add a batch dimension since
    # the TF augmentation layers operates on batched inputs.
    return tf.expand_dims(tf_image, 0)


batch = {'index':[], "image":[]}

def do_batch(batch):
    batch = preprocess_val(batch)
    batched = tf.concat([tf.expand_dims(pix, 0) for pix in batch['pixel_values']], 0)
    result = tf.nn.softmax(model({'pixel_values': batched}).logits, axis=1).numpy()
    for i, index in enumerate(batch['index']):
      df.loc[index, "classifier"] = result[i, 1]
    return {'index':[], "image":[]}

for index, row in tqdm([(index, row) for (index, row) in df.iterrows()]):=
  if row['rank'] != 'None':
    path = f'/content/drive/MyDrive/LoRA/inference/hands and palms r{row["rank"]} lora/scale 0_9/{row["side"]}_of_fair-skinned_{row["gender"]}_{row["hand"]}_hand_chkpt{row["checkpoint"]}_steps30_{row["image"]}.png'
  else:
    break
    # path = f'/content/drive/MyDrive/LoRA/inference/hands and palms r{row["rank"]} lora/scale 0_75/{row["side"]}_of_fair-skinned_{row["gender"]}_{row["hand"]}_hand_chkpt5000_steps30_{row["image"]}.png'
  image = Image.open(path)
  batch['index'].append(index)
  batch['image'].append(image)
  if len(batch['index']) >= 1:
    batch = do_batch(batch)
if batch['index']:
  batch = do_batch(batch)


100%|██████████| 576/576 [13:13<00:00,  1.38s/it]


In [ ]:
df

,rank,checkpoint,side,gender,hand,image,classifier
0,64,30000,front,male,right,5,0.058778
1,64,30000,front,male,right,4,0.832572
2,64,30000,front,male,right,3,0.791907
3,64,30000,front,male,right,2,0.851860
4,64,30000,front,male,right,1,0.725522
...,...,...,...,...,...,...,...
571,4,5000,back,female,left,4,0.528272
572,4,5000,back,female,left,3,0.847858
573,4,5000,back,female,left,2,0.083410
574,4,5000,back,female,left,1,0.878268


In [13]:
gc.open('hands and palms loras').worksheet('scale9').update('L2:L577', df[['classifier']].values.tolist())

<ipython-input-13-6cdcbb416e9e>:1: DeprecationWarning: [Deprecated][in version 6.0.0]: Method signature's arguments 'range_name' and 'values' will change their order. We recommend using named arguments for minimal impact. In addition, the argument 'values' will be mandatory of type: 'List[List]'. (ex) Worksheet.update(values = [[]], range_name=) 
  gc.open('hands and palms loras').worksheet('scale9').update('L2:L577', df[['classifier']].values.tolist())


{'spreadsheetId': '1GHwH5am7o7RKga8FPruSKLgMTXIWG_wEMKk__L0J72A',
 'updatedRange': 'scale9!L2:L577',
 'updatedRows': 576,
 'updatedColumns': 1,
 'updatedCells': 576}

In [ ]:
df

,rank,checkpoint,side,gender,hand,image,bad,ok,good,mediapipe_highconf,weighted human,mediapipe
0,64,30000,front,male,right,5,1,None,None,0.000000,0.0,0.792618
1,64,30000,front,male,right,4,1,None,None,0.000000,0.0,0.700885
2,64,30000,front,male,right,3,1,None,None,0.000000,0.0,0.972215
3,64,30000,front,male,right,2,,1,None,0.000000,0.5,0.584440
4,64,30000,front,male,right,1,1,None,None,0.000000,0.0,0.610859
...,...,...,...,...,...,...,...,...,...,...,...,...
619,None,0,back,female,left,4,1,None,None,0.994964,0.0,0.994930
620,None,0,back,female,left,3,,1,None,0.998482,0.5,0.998439
621,None,0,back,female,left,2,1,None,None,0.985213,0.0,0.980931
622,None,0,back,female,left,1,1,None,None,0.965399,0.0,0.974257
